# 필수 라이브러리 로딩

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # 스크롤
from urllib import request # 이미지
import csv
import time
# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np # 결측값 삽입용
import re # 정규표현식
import pandas as pd

# 변수 선언

In [2]:
## 교재 테이블 리스트들 
main_title = []  # 본제
sub_title = []  # 부제
author = []  # 저자
publish = []  # 출판
date = []  # 발행일
cate = []  # 네이버 카테고리
cate_kb = [] # 교보 카테고리
tot_page = []  # 전체 쪽수
size = []  # 크기
wei = []  # 무게
img = [] # 이미지
book_info = []  # 책소개
book_review = []  # 책서평
kobo_cate = []
title_isbn = [] # isbn 교재명
set_isbn = [] #set
isbn = []
book_type = [] #제본
ver = []  # 판정보
last_ver = []  # 이전 판 정보
pn_isbn = [] # 페이지 관련 끌기
keyword = []  # 키워드
price = []

index = []
idx_page = []

# 관련 사전 정의
info_dic = {'author':author, 'publish': publish, 'date':date}
isbn_dic = {'isbn':isbn, 'book_type': book_type, 'ver':ver,'last_ver':last_ver ,'pn_isbn':pn_isbn ,\
           'keyword':keyword, 'set_isbn': set_isbn, 'price':price}
plus_dic = {'book_info':book_info, 'book_review': book_review, 'index':index}

# 함수 정의

- 수정되어야 할 사항
1. 검색창에 저장된 책 제목 검색
2. 검색후 가장 위에 존재하는 판매처 클릭
> 이후 과정은 모두 동일


In [3]:
# 저자, 출판, 발행일 수집 함수
def info_func():
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_item_info__IMLyY")
    check = []
    for i in range(len(info)):
        a = info[i].text.split('\n')
        if a[0] == '저자':
            check.append('author')
            author.append(a[1]) # 저자
        elif a[0] == '출판':
            check.append('publish')
            publish.append(a[1])
        elif a[0] == '발행':
            check.append('date')
            date.append(a[1])
        else:
            pass
    for c in ['author','publish', 'date']:
        if c not in check:
            info_dic[c].append(np.NaN)

# 네이버 카테고리, isbn번호 수집  
def sub1_func():
    sub1 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_info_detail__I0Fx5") # 카테고리, ISBN
    # cate 
    cate.append(sub1[0].text)
    # code
    code = sub1[-1].text
    return code

# 페이지수, 무게, 크기 수집
def sub2_func():
    sub2 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_spec__qmQ_N") # 쪽수, 크기(무게)
    if len(sub2) == 3:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(sub2[1].text)
        # size
        size.append(sub2[2].text)
    elif len(sub2) == 2:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(np.NaN)
        # size
        size.append(sub2[1].text)
    elif len(sub2) == 1:
        tot_page.append(sub2[0].text)
        wei.append(np.NaN)
        size.append(np.NaN)
    else:
        tot_page.append(np.NaN)
        wei.append(np.NaN)
        size.append(np.NaN)

# 이미지 수집
def img_func(code):
    # image
    img_url = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[1]/div[1]/div[1]/div[1]/div/img').get_attribute("src")
    driver.implicitly_wait(5)
    request.urlretrieve(img_url,f"C:\\total_img\\{code}.jpg")
    img.append(f'{code}.jpg')

# 목차 수집시 행별 페이지수 수집
def pages(a_text):
    t_ls = []
    for x in a_text[1:-1]:
        if x.split(' ')[-1].isnumeric() == True: # 수정함
            t_ls.append(x[-1]) # sub_p
        else:
            t_ls.append(np.NaN)
    return t_ls

# 책소개, 서평, 목차 수집
def new_idx():
    plus_info = driver.find_elements(By.CLASS_NAME,'infoItem_info_item__1Nm8P')
    plus_ls = []
    for a in plus_info:
        a_text = a.text.split('\n')
        if a_text[0]== '책 소개':
            a_text = [t for t in a_text if t != '' ]
            book_info.append(a_text[1:-1])
            plus_ls.append('book_info')
            
        elif a_text[0]== '출판사 서평':
            a_text = [t for t in a_text if t != '' ]
            book_review.append(a_text[1:-1])
            plus_ls.append('book_review')
            
        elif a_text[0]== '목차':
            a_text = [t for t in a_text if t != '' ]
            index.append(a_text[1:-1])
            plus_ls.append('index')
            t_ls = pages(a_text)
            idx_page.append(t_ls)
        else:
            pass
    return plus_ls

In [4]:
# 교보문고 category 수집
def kobo():
    driver.find_element(By.PARTIAL_LINK_TEXT, '교보').click()
    driver.switch_to.window(driver.window_handles[2]) 
    driver.implicitly_wait(5)
    try:
        kb = driver.find_element(By.CLASS_NAME, 'category_list_item').text
        kb = kb.split('>')
        kb = [k.strip() for k in kb]
        cate_kb.append(list(set(kb)))
    except:
        cate_kb.append(np.NaN)
    driver.close()
    driver.implicitly_wait(5)
    driver.switch_to.window(driver.window_handles[1])
    driver.implicitly_wait(5)
    print('정보수집kb')

In [5]:
# 네이보 도서내에서 실행될 함수
def naver():
    # 창 전환
    driver.switch_to.window(driver.window_handles[1]) 
    driver.implicitly_wait(5)
    try: # 제목 수집 -부제가 없는지 확인먼저 
        sub_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text) # 부
        main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text) # 본
        
    except:
        main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text) #부제랑 안나눠졌을시
        sub_title.append(np.NaN)
    
    driver.implicitly_wait(5)
    try:
        info_func() # 저자, 출판, 발행일 수집
    except:
        for x in [author, publish, date]:
            x.append(np.NaN)  # 해당 정보가 없으면 널 값

    try:
        code = sub1_func() # 카테고리, isbn 수집
    except:
        print('sub1함수 예외상황')
    
    try:
        sub2_func() # 페이지, 책 규격
    except:
        print('sub2함수 예외상황')
    
    try:
        img_func(code) # 이미지
    except:
        img.append(np.nan)
    # 교보문고 링크 존재여부 확인   
    book_text = driver.find_elements(By.CLASS_NAME, 'bookSeller_mall_name__S0crJ')
    book_text = [bt.text.split('\n')[0] for bt in book_text]
    if '교보문고' in book_text:
        kobo() # 교보문고 카테고리 수집
    else:
        cate_kb.append(np.NaN)    
        
    plus_ls = new_idx() # 소개, 서평, 목차 수집
    for p in ['book_info','book_review','index']:
        if p not in plus_ls:
            plus_dic[p].append(np.NaN) # 수집된 결과 없을시 널 값 부여
    
    
    return code

In [6]:
# 단일권의 ISBN 수집 코드
def one_isbn():
    check = []
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    driver.implicitly_wait(5)
    tit = driver.find_element(By.CSS_SELECTOR,'#contents > div > div.resultViewDetail > div.tit').text
    title_isbn.append(tit.split(']')[1]) # 타이틀 수집, 현재는 제외된 부분
    # 행 수 카운트
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            check.append('isbn')
            isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) # 저자
        elif a[0] in ['세트ISBN','SET ISBN']:
            check.append('set_isbn')
            set_isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0]))
        elif a[0] in ['제본형식','제본형식(종이)']:
            check.append('book_type')
            book_type.append(a[-1])
        elif a[0] == '판사항':
            check.append('ver')
            ver.append(a[-1])
        elif a[0] == '이전판 정보':
            check.append('last_ver')
            last_ver.append(re.sub(r'[^0-9]', '', a[1]))
        elif a[0] == '형태 및 본문언어':
            check.append('pn_isbn')
            pn_isbn.append(np.NaN)
           # pn_isbn.append(a[1]) 세트의 개별권만 수집하도록 변경됨
        elif a[0] == '키워드':
            check.append('keyword')
            keyword.append(a[1]) 
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price.append(re.sub(r'[^0-9]', '', a[1]))
                check.append('price')
            else:
                price.append(np.NaN)
                check.append('price')
        else:
            pass

    for c in ['isbn','book_type','ver','last_ver','pn_isbn','keyword', 'set_isbn', 'price']:
        if c not in check:
            isbn_dic[c].append(np.NaN)
        else:
            pass

In [7]:
# 세트권의 각 항목별 수집해야하는 내용 모음
def set_code_search(x):
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{x}]/div/div/div/a').click()
    driver.implicitly_wait(5)
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    check = []
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            check.append('isbn')
            isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) 
        elif a[0] in ['세트ISBN','SET ISBN']:
            check.append('set_isbn')
            set_is = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
            set_isbn.append(set_is)
        elif a[0] in ['제본형식','제본형식(종이)'] :
            check.append('book_type')
            book_type.append(a[-1])
        elif a[0] == '판사항':
            check.append('ver')
            ver.append(a[-1])
        elif a[0] == '이전판 정보':
            check.append('last_ver')
            last_ver.append(re.sub(r'[^0-9]', '', a[1]))
        elif a[0] == '형태 및 본문언어':
            check.append('pn_isbn')
            pni = a[1].split('/')[0]
            pn_isbn.append(re.sub(r'[^0-9]', '', pni))
        elif a[0] == '키워드':
            check.append('keyword')
            keyword.append(a[-1])
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price.append(re.sub(r'[^0-9]', '', a[1]))
                check.append('price')
            else:
                price.append(np.NaN)
                check.append('price')

    for c in ['book_type','ver','last_ver','pn_isbn','keyword', 'price']:
        if c not in check:
            isbn_dic[c].append(np.NaN)
    if 'isbn' not in check:
        isbn.append(set_is)
    if 'set_isbn' not in check:
        set_isbn.append(set_is)
    driver.back()
    driver.implicitly_wait(5)

# 세트로 이루어진 정보들중 세트와 개별권 구별 함수
def set_code():
    find_set = driver.find_elements(By.CLASS_NAME, 'dot-list')
    last_idx = []
    first_idx = ''
    for i, fs in enumerate(find_set):
        fs = fs.text.split('\n')
        fs = [f.replace(':', '') for f in fs]
        fs = [f.split(' ')[0] for f in fs]
        print(fs)
        if 'ISBN' not in fs:
            first_idx = i+1
        else:
            last_idx.append(i+1)
    return first_idx,last_idx
 
# 위에서 구한 정보를 토대로 세트권 정보 먼저 수집, (네이버 정보와 연결하기 위함)
def set_code_find(x,y):
    set_code_search(x)
    print('set ok')
    for i in y:
        set_code_search(i)
        for c in [book_info, img, wei, size, tot_page, cate, date, publish, author, sub_title,\
                  main_title, index, idx_page, book_review, cate_kb]:
            c.append(np.NaN)
    print('last ok')

In [8]:
# isbn이 단일권인지 세트권인지 판별하는 함수
def search_isbn(code): 
    driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
    driver.implicitly_wait(5)
    num = driver.find_elements(By.CLASS_NAME, 'resultData')
    if len(num) == 1: # 단일 경우 함수
        one_isbn()
    else:
        x, y = set_code()
        set_code_find(x,y)
    driver.close() # 종료
    driver.implicitly_wait(5)
    driver.switch_to.window(driver.window_handles[0])
    print('isbn수집 완')

In [9]:
# 판매처 링크를 보유한 상품 판별 및 클릭 함수
def find_new():
    try:
#         elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        for i,ele in enumerate(elements):
            print(i, ele.text)
            ele.send_keys(Keys.ENTER)
            driver.implicitly_wait(5)
            print('상품클릭')
            # 네이버 함수
            code = naver()
            print(code)
            # isbn 함수
            search_isbn(code)
    except:
        print("find예외")
        driver.implicitly_wait(5)

In [14]:
# 검색할 단어 집합
questions = [9791168809055, 9791168809352, 9788963367620, 9791166871085, 9791192930015, 9791168809697,\
            9791165233808, 9791166184871, 9791168809154, 9791168809055, 9791165233822, 9791166871078,\
            9791160492538, 9791160492521, 9791168809642, 9791168809659, 9791168808713, 9791168809567,\
            9791168808720]

def find_flus():
    try:
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        elements[0].send_keys(Keys.ENTER)
        driver.implicitly_wait(5)
        print('상품클릭')
        # 네이버 함수
        code = naver()
        print(code)
        # isbn 함수
        search_isbn(code)
    except:
        print("find예외")
        driver.implicitly_wait(5)

In [28]:
# 검색할 단어 집합
questions = [ 9791168809659, 9791168808713, 9791168809567,\
            9791168808720]

def find_flus():
    try:
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        elements[0].send_keys(Keys.ENTER)
        driver.implicitly_wait(5)
        print('상품클릭')
        # 네이버 함수
        code = naver()
        print(code)
        # isbn 함수
        search_isbn(code)
    except:
        print("find예외")
        driver.implicitly_wait(5)

# 드라이버 세팅

In [15]:
# 첫화면
URL = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005981&pageIndex=1&pageSize=40&sort=DATE'
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

sv = Service(executable_path= 'C:\Data\chromedriver')
driver = webdriver.Chrome(service=sv, options=ops)

driver.get(url=URL)

# 데이터 수집
- 중간에 클릭 오류가 왕왕 발생해서 50개씩 끊어서 실행함
- find함수의 예외사항을 수정한다면, 한번에도 가능하지 않을까 생각함

In [24]:
ques = driver.find_element(By.CLASS_NAME, 'searchInput_search_input__vLBeq')

In [25]:
ques.send_keys('1')

In [29]:
for q in questions:
    ques = driver.find_element(By.CLASS_NAME, 'searchInput_search_input__vLBeq')
    ques.clear() # input 초기화
    ques.send_keys(q) # 검색단어 보내기
    ques.send_keys(Keys.ENTER)
    find_flus()
    

1 상품수
상품클릭
정보수집kb
9791168809659
isbn수집 완
1 상품수
상품클릭
정보수집kb
9791168808713
isbn수집 완
1 상품수
상품클릭
정보수집kb
9791168809567
isbn수집 완
1 상품수
상품클릭
정보수집kb
9791168808720
isbn수집 완


In [30]:
len(main_title), len(date), len(cate_kb), len(isbn), len(index)

(19, 19, 19, 19, 19)

In [34]:
driver.close()

# 데이터 저장

In [31]:
test = pd.DataFrame({'main_title': main_title, 'sub_title':sub_title,'cate':cate, 'tot_page':tot_page,\
                     'author':author,'publish':publish, 'date':date, 'size':size, 'wei':wei,'img':img,\
                    'book_info':book_info, 'book_review':book_review,'index':index,'idx_page':idx_page,\
                     'set_isbn':set_isbn, 'isbn':isbn,'price':price, 'book_type':book_type, 'ver':ver,\
                     'last_ver':last_ver,'pn_isbn':pn_isbn,'keyword':keyword, 'cate_kb':cate_kb})
test.to_excel('C:/police_data/total_book6-2.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [32]:
# 카테고리 수집
cate_test = pd.DataFrame({'main_title': main_title, 'isbn':isbn, 'keyword': keyword,'cate_kb':cate_kb})
cate_test.to_excel('C:/police_data/total_cate6-2.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [33]:
# 목차 테이블
idx_test = pd.DataFrame({'main_title': main_title,'isbn':isbn,'index': index, 'idx_page':idx_page})
idx_test.to_excel('C:/police_data/total_idx6-2.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
